In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from api_keys import g_key

In [17]:
# get that csv, then add heat map. 
cities = pd.read_csv("output_data/cities.csv", encoding="utf-8")
cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,London,51.5085,-0.1257,54.70,81,100,3.00,GB,1624344453
1,Paris,48.8534,2.3488,64.26,91,90,3.44,FR,1624344624
2,Chicago,41.8500,-87.6500,58.03,74,90,3.40,US,1624344307
3,Boston,42.3584,-71.0598,74.61,89,40,1.01,US,1624344363
4,Buenos Aires,-34.6132,-58.3772,53.96,91,90,4.61,AR,1624344633
5,Cairo,30.0626,31.2497,84.96,30,0,5.75,EG,1624344645
6,Algiers,36.7525,3.0420,69.62,60,0,3.44,DZ,1624344846
7,Sydney,-33.8679,151.2073,60.64,85,20,1.01,AU,1624344798
8,Melbourne,28.0836,-80.6081,75.96,91,1,4.61,US,1624344826
9,Queenstown,-31.8976,26.8753,56.08,43,5,9.62,ZA,1624344870


In [13]:
humidity = cities["Humidity"].astype(float)
print(humidity)
maxhumidity = humidity.max()
print(maxhumidity)
locations = cities[["Lat", "Lng"]]
print(locations)

0     81.0
1     91.0
2     74.0
3     89.0
4     91.0
5     30.0
6     60.0
7     85.0
8     91.0
9     43.0
10    87.0
Name: Humidity, dtype: float64
91.0
        Lat       Lng
0   51.5085   -0.1257
1   48.8534    2.3488
2   41.8500  -87.6500
3   42.3584  -71.0598
4  -34.6132  -58.3772
5   30.0626   31.2497
6   36.7525    3.0420
7  -33.8679  151.2073
8   28.0836  -80.6081
9  -31.8976   26.8753
10  42.2506  -75.8330


In [14]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
 # 10 cities and do narrow down  between 80-90 
    # each cities coordinates
narrowed_city_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 95) & (cities["Cloudiness"] >= 0)& (cities["Cloudiness"] < 90)&(cities["Wind Speed"] < 10), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Boston,42.3584,-71.0598,74.61,89,40,1.01,US,1624344363
1,Cairo,30.0626,31.2497,84.96,30,0,5.75,EG,1624344645
2,Melbourne,28.0836,-80.6081,75.96,91,1,4.61,US,1624344826


In [39]:
narrowed_city_df.loc[0]['Lat']

42.3584

In [42]:
# fig = gmaps.figure(confi )
# once you created u don't nd to recreate redo it
hotel_list=[]
for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']
    
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    json_response = requested.json()
    try:
        hotel_list.append(json_response['results'][0]['name'])
    except:
        hotel_list.append("")
narrowed_city_df["Hotel Name"] = hotel_list
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Boston,42.3584,-71.0598,74.61,89,40,1.01,US,1624344363,Boston
1,Cairo,30.0626,31.2497,84.96,30,0,5.75,EG,1624344645,Cairo
2,Melbourne,28.0836,-80.6081,75.96,91,1,4.61,US,1624344826,Palm Bay


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]

In [44]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))